- Changing the NotebookApp.iopub_data_rate_limit from 1,000,000 to 10,000,000,000 by using:
    a) jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
        1. Note: As of now, this means that jupyter needs to be launched in this manner everytime i run this
           this code in order to prevent the error. It appears theres away to change the config file though,
           so this can be looked into at another time
- Important source: https://stmorse.github.io/journal/espn-fantasy-v3.html
- Another solid source here: http://www.gregreda.com/2015/02/15/web-scraping-finding-the-api/
- I'll probably need to create a manual mapping between teamId and Team Name since they don't provide a good one in the base URL
    a) note that this aligns with the order found in the "Members" selection from the "League" drop-down
    b) This will mapping will need to be kept at a season/team level to account teams dropping in and out over time (don't worry about this now though...)

- mMatchup view
    a) schedule List (the following represents one element in the list)
      1. home and away Dictionary
          - 'cumulativeScore' Dictionary
              a) Appears to be a score breakdown by stat via the 'scoreByStat' key (returns another dictionary),
                 but it's hard to tell what each of its keys represent
          - 'gamesPlayed' Integer
             a) Seems like it's supposed to be games played but this is always 0?
          - 'pointsByScoringPeriod' Dictionary
              a) Current week is the key which returns a float of the score
              b) This is what we're looking for, but 'totalPoints' is more straightforward
          - 'teamId' Integer *** IMPORTANT KEY ***
              a) This seems to be a float for 'away' and integer for 'home'?
          - 'totalPoints' Float *** THIS IS THE PRIMARY STAT WE'RE LOOKING FOR ***
              a) points scored in that week
          

In [2]:
"""
- Changing the NotebookApp.iopub_data_rate_limit from 1,000,000 to 10,000,000,000 by using:
    a) jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
        1. Note: As of now, this means that jupyter needs to be launched in this manner everytime i run this
           this code in order to prevent the error. It appears theres away to change the config file though,
           so this can be looked into at another time
- important source: https://stmorse.github.io/journal/espn-fantasy-v3.html
"""

import requests
import pandas as pd
import numpy as np

from ratelimit import limits, sleep_and_retry

pd.options.display.max_columns = None

teamId_dict = {'2019': [
                       [1, "Connor DeLong", "DeLong"],
                       [2, "Shawn Fulford", "Fulfi"],
                       [3, "Kevin Perelstein", "K-Train"],
                       [4, "Mario Bynum", "Mario"],
                       [5, "Brendan Elliot", "Brendan"],
                       [6, "Josh Nadeau", "Germany"],
                       [7, "Kieth Kaplan", "Kieth"],
                       [8, "Jonathan Kaunert", "Jon"],
                       [9, "Brock Corsi", "Brock"], 
                       [10, "Travis Hohman", "Travis"],
                       [11, "Keanu Hines", "Keanu"],
                       [12, "Matt Fleisher", "Fleish"]
                       ],
                '2020': [
                       [1, "Connor DeLong", "DeLong"],
                       [2, "Shawn Fulford", "Fulfi"],
                       [3, "Kevin Perelstein", "K-Train"],
                       [4, "Mario Bynum", "Mario"],
                       [5, "Brian Solomon", "Solomon"],
                       [6, "Josh Nadeau", "Germany"],
                       [7, "Kieth Kaplan", "Kieth"],
                       [8, "Jonathan Kaunert", "Jon"],
                       [9, "Alex Darr", "Darr"],
                       [10, "Travis Hohman", "Travis"],
                       [11, "Keanu Hines", "Keanu"],
                       [12, "Matt Fleisher", "Fleish"]
                       ]
              }


@sleep_and_retry
@limits(calls=6000, period=600)
def pull_data(year, league_id = 0, dict_params = {}):
    """ Returns a JSON object containing the data pulled APIs url """
    if league_id == 0:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + str(year)
    else:
        if year < 2020:
            url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
                  str(league_id) + "?seasonId=" + str(year)
        else:
            url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + \
                  str(year) + "/segments/0/leagues/" + str(league_id)     
        
    r = requests.get(url, params=dict_params)
    
    status_code = r.status_code
    
    if r.status_code == 200:
        pass
    else:
        if r.status_code == 429:
            print("429 error")

        return None   
    
    # 2020 url returns JSON object while prior years return it in a list 
    if year < 2020:
        d = r.json()[0]
    else:
        d = r.json()
        
    r.close()
        
    return d

# df = pd.DataFrame(teamId_dict, columns = ['teamId', 'full_name', 'manual_nick_name'])
team_df_2019 = pd.DataFrame(teamId_dict['2019'], columns = ['teamId', 'full_name', 'manual_nickname'])
team_df_2019['seasonId'] = 2019

team_df_2020 = pd.DataFrame(teamId_dict['2020'], columns = ['teamId', 'full_name', 'manual_nickname'])
team_df_2020['seasonId'] = 2020

team_df_all_seasons = pd.concat([team_df_2019, team_df_2020])

# main_data_2019 = pull_data(2019, league_id=48347143)

# matchup_data_2019 = pull_data(2019, league_id=48347143, dict_params = {"view": "mMatchup"})
# matchup_data_2020 = pull_data(2020, league_id=48347143, dict_params = {"view": "mMatchup"})

general_data = pull_data(2019, league_id=48347143)
new = pull_data(2019, league_id=48347143)
# mMatchup = pull_data(2020, league_id=48347143, dict_params = {"view": "mMatchup"})
# mMatchup_week8 = pull_data(2020, league_id=48347143, dict_params = {"view": "mMatchup",
#                                                                    'scoringPeriodId': 8})


# mMatchupScore = pull_data(2020, league_id=48347143, dict_params = {"view": "mMatchupScore"})

# player_info_2020 = pull_data(2019, league_id=48347143, dict_params = {"view": "kona_player_info"})
# pro_team_info_2020 = pull_data(2020, dict_params = {"view": "proTeamSchedules_wl"})

# print(team_schedules_2020)
print(general_data.keys())
print(len(general_data['members']))

dict_keys(['gameId', 'id', 'members', 'scoringPeriodId', 'seasonId', 'segmentId', 'settings', 'status', 'teams'])
12


In [14]:

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

data = []
print('Week ', end='')
for week in range(1, 17):
    print(week, end=' ')
    
    request_params = {"view": "mMatchup"
                      , "view":"mMatchupScore", 
                      "scoringPeriodId": week
                     }
    request_params = (("view", "mMatchup"), ("view", "mMatchupScore"), ("scoringPeriodId", week))
    
    d = pull_data(2020, league_id=48347143, dict_params = request_params)
    
    for tm in d['teams']:
        tmid = tm['id']
        for p in tm['roster']['entries']:
            name = p['playerPoolEntry']['player']['fullName']
            slot = p['lineupSlotId']
            pos  = slotcodes[slot]

            # injured status (need try/exc bc of D/ST)
            inj = 'NA'
            try:
                inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # projected/actual points
            proj, act = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != week:
                    continue
                if stat['statSourceId'] == 0:
                    act = stat['appliedTotal']
                elif stat['statSourceId'] == 1:
                    proj = stat['appliedTotal']

            data.append([
                week, tmid, name, slot, pos, inj, proj, act
            ])
print('\nComplete.')

data = pd.DataFrame(data, 
                    columns=['Week', 'Team', 'Player', 'Slot', 
                             'Pos', 'Status', 'Proj', 'Actual'])

data

Week 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
Complete.


,Week,Team,Player,Slot,Pos,Status,Proj,Actual
0,1,1,Alvin Kamara,2,RB,ACTIVE,17.406407,21.2
1,1,1,Travis Kelce,6,TE,ACTIVE,14.886366,14.0
2,1,1,Chris Carson,2,RB,QUESTIONABLE,14.635647,21.6
3,1,1,Calvin Ridley,4,WR,QUESTIONABLE,11.783908,32.4
4,1,1,Keenan Allen,23,Flex,ACTIVE,11.671297,5.7
...,...,...,...,...,...,...,...,...
3387,16,12,Corey Davis,20,Bench,ACTIVE,NaN,NaN
3388,16,12,Cordarrelle Patterson,20,Bench,ACTIVE,NaN,NaN
3389,16,12,Panthers D/ST,20,Bench,NA,NaN,NaN
3390,16,12,Vikings D/ST,16,Def,NA,NaN,NaN


In [12]:
# from random import randint

# value = randint(10000000, 99000000)

# print(value, value + 30000)

# request_params = {"view": "mMatchup"
# #                   , "view":"mMatchupScore"
#                   , "scoringPeriodId": week
#                  }

request_params = (('view', 'mMatchup'), ('view', 'mMatchupScore'), ('scoringPeriodId', 5))

d = pull_data(2020, league_id=48347143, dict_params=request_params)

# league_id = 48347143
# season    = 2020

# url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
#       str(season) + '/segments/0/leagues/' + str(league_id) + \
#       '?view=mMatchup&view=mMatchupScore' 

# r = requests.get(url,
#                  params={'scoringPeriodId': 5})
# d = r.json()

print(d.keys())

dict_keys(['draftDetail', 'gameId', 'id', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'status', 'teams'])


In [5]:
# Use the matchup period portion below to get the upcoming week number:
# for i in range(len(mMatchup['schedule'])):
#     print(mMatchup['schedule'][i]['home'].keys(), "AND ", 
#           mMatchup['schedule'][i]['matchupPeriodId'],
#           '\n')

file_dir = '/home/cdelong/python_projects/ff_web_app/delt_ff_standings/Leagues_Found.csv'

def find_leagues(startId, endId, seasonId):
    active_leagues = []

    for leagueId in range(startId, endId):   
        general_league_data = pull_data(seasonId, league_id=leagueId)
        
        columns = ['leagueId', 'number_of_teams', 'startId', 'endId', 'seasonId', 'status_code']            

        if general_league_data == None:
            pass
        else:
            num_teams = len(general_league_data['members'])

            current_league = [leagueId, num_teams, startId, endId, seasonId, 200]
            
            df_current_league = pd.DataFrame(data=[current_league], columns=columns)
            df_current_league.to_csv(file_dir, mode='a', header=False)
            
            active_leagues.append(current_league)
            
    df_current_league = pd.DataFrame(data=active_leagues, columns=columns)
            
    return df_current_league

for_start = 58547291

get_leagues = find_leagues(for_start, for_start + 100000, 2020)
get_leagues

,leagueId,number_of_teams,startId,endId,seasonId,status_code
0,58547293,6,58547291,58647291,2020,200
1,58547742,12,58547291,58647291,2020,200
2,58547769,10,58547291,58647291,2020,200
3,58547940,12,58547291,58647291,2020,200
4,58548053,12,58547291,58647291,2020,200
...,...,...,...,...,...,...
492,58646442,10,58547291,58647291,2020,200
493,58646567,10,58547291,58647291,2020,200
494,58646855,13,58547291,58647291,2020,200
495,58646879,10,58547291,58647291,2020,200


In [22]:
request_params = (('view', 'mMatchup'), ('view', 'mMatchupScore'), ('scoringPeriodId', 6))

mMatchup_week8 = pull_data(2020, league_id=48347143, dict_params = request_params)

print(mMatchup_week8['scoringPeriodId'])

for i in range(len(mMatchup_week8['schedule'])):
    print(mMatchup_week8['schedule'][i]['home'].keys(), "AND ", 
          mMatchup_week8['schedule'][i]['matchupPeriodId'],
          '\n')
    
# general_data = pull_data(2019, league_id=48347143)
# print(len(general_data['members']))

6
dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  1 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  1 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  1 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  1 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  1 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  1 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'tiebreak', 'totalPoints']) AND  2 

dict_keys(['adjustment', 'cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod

In [13]:
print(mMatchup.keys(), "\n")
print(mMatchup['scoringPeriodId'], "\n")
print(mMatchup['teams'][0]['roster'].keys(), "\n")
print(len(mMatchup['teams'][0]), "\n")
# print(mMatchupScore.keys(), "\n")

print(mMatchup['schedule'][0].keys(), "\n")
print(mMatchup['schedule'][0]['away'].keys(), "\n")
# print(mMatchup['schedule'][30]['away']['pointsByScoringPeriod'])
# print(mMatchup['schedule'][30]['away']['totalPoints'])
print(mMatchup['schedule'][30]['away']['pointsByScoringPeriod'].keys(), "\n")


print(mMatchup['schedule'][30]['away']['cumulativeScore']['wins'], "\n")
# print(mMatchup['schedule'][30]['away']['cumulativeScore']['scoreByStat'])
# for check in mMatchup['schedule'][30]['away']['cumulativeScore']['scoreByStat'].items():
#     print(check)

for i in range(len(mMatchup['schedule'])):
    print(mMatchup['schedule'][i]['home'].keys(), "AND ", 
          mMatchup['schedule'][i]['matchupPeriodId'],
          '\n')

dict_keys(['draftDetail', 'gameId', 'id', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'status', 'teams']) 

14 

dict_keys(['appliedStatTotal', 'entries']) 

2 

dict_keys(['away', 'home', 'id', 'matchupPeriodId', 'winner']) 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) 

dict_keys(['6']) 

0 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) AND  1 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) AND  1 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) AND  1 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) AND  1 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) AND  1 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) AND  1 

dict_keys(

In [127]:
mMatchup = pull_data(2020, league_id=48347143, dict_params = {"view": "mMatchup", 'scoringPeriodId': 1})

print(mMatchup['teams'][0].keys(), '\n')
print(len(mMatchup['teams']), '\n')

print(mMatchup['teams'][0]['roster'], '\n')


# for i in range(len(mMatchupScore['schedule'])):
#     print(mMatchupScore['schedule'][i]['home'].keys(), "AND ", i, '\n')


print(mMatchupScore['schedule'][72]['home'].keys(), '\n')

roster = mMatchupScore['schedule'][74]['home']['rosterForCurrentScoringPeriod']['entries']

print(mMatchupScore['schedule'][74]['home']['totalPointsLive'], '\n')

for check in roster[0]['playerPoolEntry']['player']['stats']:
    print(check,'\n')

# for i in range(len(roster)):
#     print(roster[i].keys())
# print(len(mMatchupScore['schedule'][74]['away']['rosterForCurrentScoringPeriod']['entries']), '\n')
# print(mMatchupScore['schedule'][30]['away'].keys(), "\n")

# print(mMatchupScore['schedule'][30]['away']['cumulativeScore'], "\n")
# print(mMatchup['schedule'][30]['away']['totalPoints'], "\n")
# print(mMatchupScore['schedule'][30]['away']['totalPoints'], "\n")

dict_keys(['id', 'roster']) 

12 

{'appliedStatTotal': 130.5, 'entries': [{'acquisitionDate': None, 'acquisitionType': None, 'injuryStatus': 'NORMAL', 'lineupSlotId': 2, 'pendingTransactionIds': None, 'playerId': 3054850, 'playerPoolEntry': {'appliedStatTotal': 21.200000000000003, 'id': 3054850, 'keeperValue': 1, 'keeperValueFuture': 1, 'lineupLocked': True, 'onTeamId': 1, 'player': {'active': True, 'defaultPositionId': 2, 'droppable': False, 'eligibleSlots': [2, 3, 23, 7, 20, 21], 'firstName': 'Alvin', 'fullName': 'Alvin Kamara', 'id': 3054850, 'injured': False, 'injuryStatus': 'ACTIVE', 'lastName': 'Kamara', 'lastNewsDate': 1607116825000, 'lastVideoDate': 1574613040000, 'proTeamId': 18, 'rankings': {'0': [{'auctionValue': 0, 'rank': 0, 'rankSourceId': 0, 'rankType': 'STANDARD', 'slotId': 2}, {'auctionValue': 0, 'rank': 0, 'rankSourceId': 0, 'rankType': 'PPR', 'slotId': 2}, {'auctionValue': 0, 'rank': 4, 'rankSourceId': 2, 'rankType': 'PPR', 'slotId': 2}, {'auctionValue': 0, 'rank': 

In [110]:
# mMatchup_new = pull_data(2020, league_id=48347143, dict_params = {"view": "mMatchup", 'scoringPeriodId': '8'})
print(mMatchup_new['schedule'][0].keys(), '\n')
print(len(mMatchup_new['schedule']), '\n')
print(mMatchup_new['schedule'][0]['away'].keys(), '\n')



dict_keys(['away', 'home', 'id', 'matchupPeriodId', 'winner']) 

78 

dict_keys(['cumulativeScore', 'gamesPlayed', 'pointsByScoringPeriod', 'teamId', 'totalPoints']) 



In [68]:
import numpy as np

choose_data = matchup_data_2020.copy()
team_df = team_df_2020.copy()

def create_matchup_df (matchup_data, playoff_week_start = 14):
    """ Returns a week/matchup level dataframe containing the total scores of each team 
        Note that this requires the mMatchup filtered data
    """
    
    data = []

    for i, matchup_dict in enumerate(matchup_data['schedule']):
    #     print(check.keys(), "AND ", choose_data['schedule'][i]['matchupPeriodId'])

        week_number = matchup_dict['matchupPeriodId']

        # bye weeks cause the "away" key to be missing for the team with a bye
        try:
            teamId_away = int(matchup_dict['away']['teamId'])
            score_away = matchup_dict['away']['totalPoints']
        except KeyError:
            teamId_away = None
            score_away = None

        # applying this here just in case
        try:
            teamId_home = int(matchup_dict['home']['teamId'])
            score_home = matchup_dict['home']['totalPoints']
        except KeyError:
            teamId_home = None
            score_home = None

        data.append([week_number, teamId_away, score_away, teamId_home, score_home])
        
    
    df_matchup = pd.DataFrame(data, columns = ['week_number', 'teamId_away', 'score_away',
                                            'teamId_home', 'score_home']
                              )
    
    df_matchup['week_type'] = ['Regular' if w<=playoff_week_start else 'Playoffs' \
                               for w in df_matchup['week_number']]

    return df_matchup

create_matchup_df(choose_data)

,week_number,teamId_away,score_away,teamId_home,score_home,week_type
0,1,5.0,100.90,3,89.70,Regular
1,1,9.0,91.64,11,120.66,Regular
2,1,1.0,119.56,7,132.02,Regular
3,1,6.0,113.30,2,96.20,Regular
4,1,8.0,74.76,12,102.26,Regular
...,...,...,...,...,...,...
92,16,8.0,118.92,5,68.96,Playoffs
93,16,7.0,75.44,11,59.26,Playoffs
94,16,2.0,128.76,4,87.60,Playoffs
95,16,10.0,62.46,3,135.82,Playoffs


In [31]:
import numpy as np

choose_data = matchup_data_2020.copy()
team_df = team_df_2020.copy()

def create_matchup_df (matchup_data, playoff_week_start = 14):
    """ Returns a week/matchup level dataframe containing the total scores of each team 
        Note that this requires the mMatchup filtered data
    """
    
    data = []

    for i, matchup_dict in enumerate(matchup_data['schedule']):
    #     print(check.keys(), "AND ", choose_data['schedule'][i]['matchupPeriodId'])

        week_number = matchup_dict['matchupPeriodId']

        # bye weeks cause the "away" key to be missing for the team with a bye
        try:
            teamId_away = int(matchup_dict['away']['teamId'])
            score_away = matchup_dict['away']['totalPoints']
        except KeyError:
            teamId_away = None
            score_away = None

        # applying this here just in case
        try:
            teamId_home = int(matchup_dict['home']['teamId'])
            score_home = matchup_dict['home']['totalPoints']
        except KeyError:
            teamId_home = None
            score_home = None

        data.append([week_number, teamId_away, score_away, teamId_home, score_home])
    
    df_matchup = pd.DataFrame(data, columns = ['week_number', 'teamId_away', 'score_away',
                                            'teamId_home', 'score_home']
                              )
    
    df_matchup['week_type'] = ['Regular' if w<=playoff_week_start else 'Playoffs' \
                               for w in df_matchup['week_number']]

    return df_matchup

def expand_matchup_data(matchup_data):
    """ Expands data to the week/team level """
    
    # create dataset from the perspective of the away team
    df_away_data = matchup_data.copy()
    df_away_data.rename(columns = {'teamId_away': 'teamId', 
                                   'score_away': 'score', 
                                   'teamId_home': 'teamId_opp',
                                   'score_home': 'score_opp'
                                  },
                        inplace=True)  
    df_away_data['home_or_away'] = 'away'
    
    # create dataset from the perspective of the home team
    df_home_data = matchup_data.copy()
    df_home_data.rename(columns = {'teamId_home': 'teamId', 
                                   'score_home': 'score', 
                                   'teamId_away': 'teamId_opp',
                                   'score_away': 'score_opp'
                                  },
                        inplace=True)  
    df_home_data['home_or_away'] = 'home'
    
    # stack data and filter out the null teamIds (this is from bye weeks)
    df_expanded = pd.concat([df_home_data, df_away_data])
    df_expanded = df_expanded[df_expanded['teamId'].notnull()]
    
    # replace null values in teamId_opp and with sentinel value (-1) 
    # so it can be converted to integer and score_opp with 0 
    df_expanded['teamId_opp'].fillna(-1, inplace=True)
    df_expanded['score_opp'].fillna(0, inplace=True)
    
    # convert the teamIds to integers
    df_expanded['teamId'] = df_expanded['teamId'].astype('int')
    df_expanded['teamId_opp'] = df_expanded['teamId_opp'].astype('int')
    
    df_expanded
    return df_expanded

def add_win_loss_ind(matchup_data):
    ''' Returns Week/Team level dataframe with win, loss, tie indicators added '''
    matchup_data = matchup_data.copy()
     # create list for the np.select parameter to create win and loss flags
    conditions = [
        (matchup_data['score'] > matchup_data['score_opp']),
        (matchup_data['score'] < matchup_data['score_opp'])
        ]
    
    matchup_data['win_ind'] = np.select(conditions, [1, 0])
    matchup_data['loss_ind'] = np.select(conditions, [0, 1])
    
    # create list for the np.select parameter to create tie flag
    conditions = [
        (matchup_data['score'] == matchup_data['score_opp']),
        (matchup_data['score'] != matchup_data['score_opp'])
        ]
    
    matchup_data['tie_ind'] = np.select(conditions, [1, 0])
    
    # create a metric that combines wins and ties (1 for win and .5 for tie)
    matchup_data['total_wins'] = matchup_data['win_ind'] + .5 * matchup_data['tie_ind']
    
    return matchup_data

def add_all_play (matchup_data):
    ''' Returns Week/Team level dataframe with all play wins and losses added '''
    matchup_data = matchup_data.copy()
    
    number_of_teams = len(matchup_data.groupby(['teamId'], as_index = False).size().index)
    
    # Get the week_number/scores freq and sort in ascending order (lowest score to highest)
    # Note that this will allow for a more robust calculation for ties given that all ties in a given
    # week will be grouped together
    get_size = matchup_data.groupby(['week_number', 'score'], as_index=False).size()
    get_size.sort_values(by=['week_number', 'score'], inplace=True, ascending=True)

    # Create the cumulative sum of the week_number/scores freq
    # (i.e. number of scores the ith score beats + the number of i scores)
    get_size['cum_sum'] = get_size.groupby(['week_number'])['size'].cumsum()

    # create variable that accounts for ties - if no ties, then 0. Otherwise, 1/number of likewise scores
    get_size['add_tie_amount'] = 0
    get_size.loc[get_size['size'] > 1, 'add_tie_amount'] = 1/get_size['size']
    get_size['all_play_wins'] = get_size['cum_sum'] - get_size['size'] + get_size['add_tie_amount'] 
    
    # merge all_play_wins onto main dataset and add all_play_losses
    matchup_data = pd.merge(matchup_data, get_size[['week_number', 'score', 'all_play_wins']], 
                                                on=['week_number', 'score'], how='outer')
    
    matchup_data['all_play_losses'] = number_of_teams - matchup_data['all_play_wins'] - 1
    matchup_data.sort_values(by=['week_number', 'teamId'], inplace=True, ascending=True)
    
    return matchup_data

def add_cum_metrics(matchup_data, cum_metrics_dict, by_group='teamId', cum_group='week_number'):
    ''' Returns Week/Team level dataframe with cumulative metrics added to it '''
    matchup_data = matchup_data.copy()
    
    # need to rethink how this by_group get passed - currently, if more than one by_group needs to be included,
    # you need to pass them all as strings separated by commas. It would make more sense for this to be
    # stored in an actual data structure (e.g. a list)
    matchup_data.sort_values(by=[by_group, cum_group], inplace=True)
    
    for cum_metric, new_col_name in cum_metrics_dict.items():
        matchup_data[new_col_name] = matchup_data.groupby([by_group])[cum_metric].cumsum()
        
    return matchup_data

def merge_on_team_data(matchup_data, team_df):
    ''' Returns Week/Team level dataframe with team information merged on '''
    matchup_data = matchup_data.copy()
    team_df = team_df.copy()
    
    matchup_data = pd.merge(matchup_data, team_df, on=['teamId'])
    matchup_data.sort_values(by=['week_number', 'teamId'], inplace=True, ascending=True)
    
    return matchup_data

def add_standings(matchup_data, week_number, 
             rank_metrics_by_week_range={'1-12': [['cum_total_wins', 'cum_score'],
                                                  [False, False]]}):
    ''' Returns one week of team level data that includes the standings for that week'''
    # filter matchup_data on the week_number of interest
    matchup_data = matchup_data.copy()
    matchup_data = matchup_data.loc[matchup_data['week_number'] == week_number]
    
    # get the number of teams and number of different ranking tiers
    number_of_teams = len(matchup_data.groupby(['teamId'], as_index = False).size().index)
    num_rank_ranges = len(rank_metrics_by_week_range.keys())
    
    # get the column names into a list and add 'rank'
    columns = list(matchup_data.columns)
    columns.append('standings')
    
    # create an empty dataframe containing all of columns from matchup_data plus 'rank'
    df_matchup_data_stacked = pd.DataFrame(columns=columns)
    
    for key, value in rank_metrics_by_week_range.items():
        # pull the lower and upper rank numbers
        rank_range_lower = int(key[0:key.find('-')].strip())
        rank_range_upper = int(key[key.find('-') + 1:].strip())
        
        # create list of all the ranks that will be calculated in the loop
        rank_list = []
        for i in range(rank_range_lower, rank_range_upper + 1):
            rank_list.append(i)
        
        # create an index number to filter out the remaining matchup data
        index_upper = rank_range_upper - rank_range_lower + 1
        
        primary_rank_metric = value[0][0]
        secondary_rank_metric = value[0][1]
        
        primary_ascending = value[1][0]
        secondary_ascending = value[1][1]
        
        # sort the remaining matchup data according to current tiers' metrics
        matchup_data.sort_values([primary_rank_metric, secondary_rank_metric], 
                                 ascending=(primary_ascending, secondary_ascending), inplace=True)
        
#         matchup_data.reset_index(inplace=True)
        df_append = matchup_data.iloc[:index_upper].copy()
        
        df_append.reset_index(inplace=True)

        # create the rank variable
        for i in range(0, index_upper):
            df_append.loc[i, 'standings'] = rank_list[i]
        
        # filter out the matchup data used for df_append so it doesn't get used in the next iteration
        if rank_range_upper < number_of_teams:
            matchup_data = matchup_data.iloc[index_upper:]
        
        df_matchup_data_stacked = df_matchup_data_stacked.append(df_append)
        
    df_matchup_data_stacked['standings'] = df_matchup_data_stacked['standings'].astype('int')
    df_matchup_data_stacked.reset_index(inplace=True)
    
    return df_matchup_data_stacked

def add_all_standings(df_matchup_data, number_of_weeks=13,
             rank_metrics_by_week_range={'1-12': [['cum_total_wins', 'cum_score'],
                                                  [False, False]]}):
    ''' Returns Week/Team level dataframe that includes week level standings'''
    df_matchup_data = df_matchup_data.copy()
    
    # get the column names into a list and add 'rank'
    columns = list(df_matchup_data.columns)
    columns.append('standings')
    
    # create an empty dataframe containing all of columns from matchup_data plus 'rank'
    df_matchup_data_stacked = pd.DataFrame(columns=columns) 

    # create the standings for each ith week and append the data to the stack dataframe
    for i in range(1, number_of_weeks + 1):

        df_matchup_data_w_standings = add_standings(df_matchup_data, i, rank_metrics_by_week_range)
        
        df_matchup_data_stacked = df_matchup_data_stacked.append(df_matchup_data_w_standings)
        
    df_matchup_data_stacked.reset_index(inplace=True, drop=True)
    
    return df_matchup_data_stacked

    
df_matchup_data = create_matchup_df(choose_data)
df_expanded_matchup = expand_matchup_data(df_matchup_data)
df_matchup_data_w_wl = add_win_loss_ind(df_expanded_matchup)
df_matchup_data_w_all_play = add_all_play(df_matchup_data_w_wl)

cum_dictionary = {'score': 'cum_score', 'win_ind': 'cum_wins', 'all_play_wins': 'cum_all_play_wins'
                  , 'all_play_losses': 'cum_all_play_losses'
                  ,'loss_ind': 'cum_losses', 'total_wins': 'cum_total_wins'}
df_matchup_data_w_cum = add_cum_metrics(df_matchup_data_w_all_play, cum_dictionary)

df_matchup_data_w_team = merge_on_team_data(df_matchup_data_w_cum, team_df)

rank_metrics_by_week_range = {'1-4': [['cum_total_wins', 'cum_score'], [False, False]]
                             ,'5-6': [['cum_all_play_wins', 'cum_score'], [False, False]]
                             ,'7-12': [['cum_total_wins', 'cum_score'], [False, False]]}
df_final = add_all_standings(df_matchup_data_w_team, rank_metrics_by_week_range=rank_metrics_by_week_range)

df_current_standings = df_final[['week_number', 'full_name', 'standings', 'cum_total_wins','score',
       'all_play_wins', 'cum_score', 'cum_all_play_wins']].loc[df_final['week_number']==3]

df_current_standings

df_matchup_data_w_all_play.loc[df_matchup_data_w_all_play['week_number']==3]

# file_dir = '/home/cdelong/python_projects/ff_web_app/\
# delt_ff_standings/weekly_standings_csvs/Delt_2020_Week2_Standings.csv'

# df_current_standings.to_csv(file_dir)

,week_number,teamId_opp,score_opp,teamId,score,week_type,home_or_away,win_ind,loss_ind,tie_ind,total_wins,all_play_wins,all_play_losses
13,3,5,108.38,1,171.32,Regular,home,1,0,0,1.0,11.0,0.0
18,3,12,127.12,2,75.14,Regular,home,0,1,0,0.0,0.0,11.0
15,3,11,101.38,3,154.08,Regular,home,1,0,0,1.0,10.0,1.0
154,3,8,89.70,4,113.98,Regular,away,1,0,0,1.0,7.0,4.0
150,3,1,171.32,5,108.38,Regular,away,0,1,0,0.0,6.0,5.0
153,3,10,90.70,6,124.00,Regular,away,1,0,0,1.0,8.0,3.0
151,3,9,87.30,7,107.04,Regular,away,1,0,0,1.0,5.0,6.0
17,3,4,113.98,8,89.70,Regular,home,0,1,0,0.0,2.0,9.0
14,3,7,107.04,9,87.30,Regular,home,0,1,0,0.0,1.0,10.0
16,3,6,124.00,10,90.70,Regular,home,0,1,0,0.0,3.0,8.0


In [38]:
# df_matchup_data_w_all_play.loc[df_matchup_data_w_all_play['week_number']==2]

# df_current_standings

check_df = df_current_standings.sort_values(by=['week_number', 'score'], inplace=False, ascending=False)

check_df

,week_number,full_name,standings,cum_total_wins,score,all_play_wins,cum_score,cum_all_play_wins
25,3,Connor DeLong,2,2.0,171.32,11.0,437.50,29.0
24,3,Kevin Perelstein,1,3.0,154.08,10.0,401.92,25.0
26,3,Matt Fleisher,3,2.0,127.12,9.0,363.74,21.0
29,3,Josh Nadeau,6,1.0,124.00,8.0,315.42,16.0
27,3,Mario Bynum,4,2.0,113.98,7.0,322.06,18.0
35,3,Brian Solomon,12,0.0,108.38,6.0,293.74,12.0
28,3,Kieth Kaplan,5,2.0,107.04,5.0,312.22,16.5
32,3,Keanu Hines,9,1.0,101.38,4.0,319.54,14.0
33,3,Travis Hohman,10,1.0,90.70,3.0,313.60,13.0
31,3,Jonathan Kaunert,8,1.0,89.70,2.0,319.64,13.0


In [20]:
def survivor_challenge(df_matchup_data, week_number):
    df_matchup_data = df_matchup_data.copy()
    
    # create a dataframe containing all of the team names
    df_remaining_teams = df_matchup_data.groupby(['full_name'], as_index=False).size()
    df_remaining_teams = df_remaining_teams['full_name']

    for week in range(1, week_number + 1):
        # create a dataframe containing the current week's data
        df_matchup_data_week = df_matchup_data.loc[df_matchup_data['week_number'] == week].copy()
        
        # merge the dataframe onto the remaining teams df so that only the teams still alive are considered
        df_matchup_data_week = pd.merge(df_remaining_teams, df_matchup_data_week, 
                                        on=['full_name'], how='left')
        
        # sort the values so that the lowest scoring team is listed at the top
        df_matchup_data_week.sort_values(by=['score', 'cum_score'], inplace=True, ascending=True)
        
        survivor_loser = df_matchup_data_week.iloc[0]
        survivor_loser = survivor_loser['full_name']
        
        df_remaining_teams = df_remaining_teams.loc[df_remaining_teams != survivor_loser]
        
    return (df_remaining_teams, survivor_loser)
    

full_data, loser = survivor_challenge(df_final, 2)

print(loser)
full_data
    

Travis Hohman


1        Brian Solomon
2        Connor DeLong
3     Jonathan Kaunert
4          Josh Nadeau
5          Keanu Hines
6     Kevin Perelstein
7         Kieth Kaplan
8          Mario Bynum
9        Matt Fleisher
10       Shawn Fulford
Name: full_name, dtype: object

In [7]:
import os
os.path.abspath(os.getcwd())

# for_csv = df_final[['week_number', 'full_name', 'standings', 'score',
#                      'cum_total_wins', 'cum_score', 'cum_all_play_wins']].loc[df_final['week_number']==1]

# df_final.to_csv(r'/home/cdelong/python_projects/personal_projects/check_2019_standings.csv')

'/home/cdelong/python_projects/ff_web_app'

In [356]:
espn_player_info = player_info_2020
espn_pro_team_info = pro_team_info_2020

def scrape_espn_player_adp(player_info, pro_team_info):
    """ Returns a player level dataframe containing the ESPN ADP of each player """
    pro_team_list = []
    for i, pro_team in enumerate(pro_team_info['settings']['proTeams']):
        pro_team_id = pro_team['id']
        pro_team_location= pro_team['location']
        pro_team_name = pro_team['name']

        temp_list = [pro_team_id, pro_team_location, pro_team_name]

        pro_team_list.append(temp_list)

    pro_team_columns = ['pro_team_id', 'pro_team_location', 'pro_team_name']

    df_pro_team = pd.DataFrame(pro_team_list, columns=pro_team_columns)

    player_info_list = []
    for i, player in enumerate(player_info['players']):
        player_teamId = player['player']['proTeamId']        
        player_fullName = player['player']['fullName']
        player_adp = player['player']['ownership']['averageDraftPosition']

        temp_list = [player_teamId, player_fullName, player_adp]

        player_info_list.append(temp_list)

    player_info_columns = ['pro_team_id', 'player_fullName', 'player_adp']
    df_player_info = pd.DataFrame(player_info_list, columns=player_info_columns)

    df_player_info = pd.merge(df_player_info, df_pro_team, on='pro_team_id', how='left')
    
    print(len(player_info['players']))
    
    return df_player_info

df_espn_player_adp = scrape_espn_player_adp(espn_player_info, espn_pro_team_info)
df_espn_player_adp

1054


,pro_team_id,player_fullName,player_adp,pro_team_location,pro_team_name
0,34,Texans D/ST,0.0,Houston,Texans
1,33,Ravens D/ST,0.0,Baltimore,Ravens
2,30,Jaguars D/ST,0.0,Jacksonville,Jaguars
3,29,Panthers D/ST,0.0,Carolina,Panthers
4,28,Washington D/ST,0.0,Washington,
...,...,...,...,...,...
1049,16,Brandon Dillon,0.0,Minnesota,Vikings
1050,18,Deonte Harris,0.0,New Orleans,Saints
1051,2,Christian Wade,0.0,Buffalo,Bills
1052,1,Craig Reynolds,0.0,Atlanta,Falcons


In [325]:
import urllib.request
from bs4 import BeautifulSoup as bSoup

class Scraper:
    def __init__(self, site):
        self.site = site
        
        r = urllib.request.urlopen(self.site)
        parser = "html.parser"
        
        self.html = r.read()
        self.sp = bSoup(self.html, parser)       
        
    def scrape(self):
        r = urllib.request.urlopen(self.site)
        html = r.read()
        parser = "html.parser"
        sp = bSoup(html, parser)

def scrape_yahoo_player_adp(yahoo_adp_page):
    """ Returns a dataframe containing the ADP player from one page (50 players)"""
    
    # create a tag object for the parent tag containing the ADP table data
    dt_parent_tag = yahoo_adp_page.sp.find("table", id='draftanalysistable')

    # create a tag object of the body of the dt_parent_tag.contents list (note that there are only 2 elements
    # in this - the first contains everything under the "thead" tag)
    dt_body_tag = dt_parent_tag.contents[1]

    all_player_data_list = []
    for i, tr_tag in enumerate(dt_body_tag.contents):
        player_data_list = []
        for k, td_tag in enumerate(tr_tag):
            # "descendents" attribute returning NoneType for some reason?
            # tusing this should be a better way of scraping the name and team than the shit below
            if k == 0:
                # get the player name
                player_data = td_tag.contents[0].contents[1].contents[3].contents[0].contents[0]
                player_data_list.append(player_data)

                # get the player's team and positino
                player_data = td_tag.contents[0].contents[1].contents[3].contents[2].contents[0]
                player_data_list.append(player_data)
            else:
                player_data = td_tag.contents[0].contents[0]
                player_data_list.append(player_data)

        all_player_data_list.append(player_data_list)

    # convert the numeric metrics to float
    for player in all_player_data_list:
        for i, data in enumerate(player[2:]):
            player[i + 2] = float(data.replace("%", ""))
            
        
    columns = ['player_fullName', 'player_team_position', 'player_adp', 
               'player_avg_round', 'player_perc_drafted']
    
    df_all_player_data = pd.DataFrame(all_player_data_list, columns=columns)
    
    return df_all_player_data

def compile_all_yahoo_adp(page_numbers_list, base_url = 
                "https://football.fantasysports.yahoo.com/f1/draftanalysis?tab=SD&pos=ALL&sort=DA_AP&count="):
    """ Returns a dataframe containing the ADP data from the pages in the 'page_numbers_list' param """
    
    # create a player ADP dataframe for each page number in the list and append it to an overall dataframe
    for i, page_number in enumerate(page_numbers_list):
        url = base_url + str(page_number)
        yahoo_adp_page = Scraper(url)
        df_yahoo_player_adp = scrape_yahoo_player_adp(yahoo_adp_page)
        
        if i == 0:
            # get the columns in the player adp dataframe and create and empty dataframe for stacking
            columns = list(df_yahoo_player_adp.columns)
            df_all_yahoo_player_adp = pd.DataFrame(columns=columns)
            

        df_all_yahoo_player_adp = df_all_yahoo_player_adp.append(df_yahoo_player_adp)
  
    df_all_yahoo_player_adp.reset_index(inplace=True)
    
    return df_all_yahoo_player_adp

page_numbers_list = [0, 50, 100, 150, 200]
df_all_yahoo_player_adp = compile_all_yahoo_adp(page_numbers_list)
df_all_yahoo_player_adp


,index,player_fullName,player_team_position,player_adp,player_avg_round,player_perc_drafted
0,0,Christian McCaffrey,Car - RB,1.1,1.0,100.0
1,1,Saquon Barkley,NYG - RB,2.3,1.0,100.0
2,2,Ezekiel Elliott,Dal - RB,3.6,1.0,100.0
3,3,Alvin Kamara,NO - RB,4.8,1.0,100.0
4,4,Derrick Henry,Ten - RB,5.6,1.0,100.0
...,...,...,...,...,...,...
224,24,Los Angeles,LAC - DEF,140.3,14.9,59.0
225,25,Joey Slye,Car - K,140.7,14.7,12.0
226,26,Tennessee,Ten - DEF,141.2,14.9,24.0
227,27,Cleveland,Cle - DEF,142.2,15.0,12.0


In [348]:
player_info_2020 = pull_data(2020, league_id=48347143, dict_params = {"view": "kona_player_info"})
pro_team_info_2020 = pull_data(2020, dict_params = {"view": "proTeamSchedules_wl"})

df_all_yahoo_player_adp = compile_all_yahoo_adp(page_numbers_list)
df_espn_player_adp = scrape_espn_player_adp(player_info_2020, pro_team_info_2020)

df_all_yahoo_player_adp.drop('index', 1,  inplace=True)

df_all_yahoo_player_adp.to_csv(r'/home/cdelong/python_projects/personal_projects/yahoo_adp.csv', index=False)
df_espn_player_adp.to_csv(r'/home/cdelong/python_projects/personal_projects/espn_adp.csv', index=False)

df_espn_player_adp

50


,pro_team_id,player_fullName,player_adp,pro_team_location,pro_team_name
0,16,Ameer Abdullah,169.988215,Minnesota,Vikings
1,9,Davante Adams,17.011242,Green Bay,Packers
2,0,Josh Adams,169.979466,,FA
3,0,Rodney Adams,169.991083,,FA
4,0,Quincy Adeboyejo,169.976556,,FA
5,13,Nelson Agholor,170.007362,Las Vegas,Raiders
6,8,Jamal Agnew,169.993704,Detroit,Lions
7,0,Salvon Ahmed,169.991893,,FA
8,0,Kamar Aiken,169.996206,,FA
9,25,Brandon Aiyuk,166.892858,San Francisco,49ers


In [11]:
# Example from https://realpython.com/python-web-scraping-practical-introduction/

from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)


In [317]:
df_yahoo_player_adp = scrape_yahoo_player_adp(yahoo_adp_page)
list(df_yahoo_player_adp.columns)

['player_fullName',
 'player_team_position',
 'player_adp',
 'player_avg_round',
 'player_perc_drafted']

Website for Chromedriver install is located here: https://yizeng.me/2014/04/20/install-chromedriver-and-phantomjs-on-linux-mint/

A couple things to note:
    - Based on the chromedriver documentation, the version you're installing needs to align with the version of chrome you have. This means the "2.29" portion of the url path in the "wget" command needs to be updated accordingly
    - Rather than usig the "wget" command, you can just download the desired version directly from the following site: https://chromedriver.chromium.org/downloads
    - located in "/usr/local/share"

In [136]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

url = "http://www.python.org"
url = "https://football.fantasysports.yahoo.com/f1/draftanalysis"

driver = webdriver.Chrome("/usr/local/share/chromedriver")
driver.get(url)


In [349]:
check = matchup_data_w_cum.copy()

# check2 = check.pivot_table(index=['teamId','week_number'], aggfunc='size')
check2 = check.groupby(['teamId_opp'], as_index=False).size()
check2['teamId_opp'] = check2['teamId_opp'].astype(int)

check['teamId_opp'].fillna(-1, inplace=True)
check['teamId_opp'] = check['teamId_opp'].astype('int')

# print(check.loc[check['teamId_opp'] == Null, ['teamId', 'week_number']])
print(check.loc[check['week_number'] == 14, ['teamId', 'teamId_opp', 'week_number']])

print(check2)

    teamId  teamId_opp  week_number
81     1.0          -1           14
84     2.0           9           14
83     3.0          10           14
82     4.0           6           14
80     5.0           7           14
82     6.0           4           14
80     7.0           5           14
79     8.0          11           14
84     9.0           2           14
83    10.0           3           14
79    11.0           8           14
78    12.0          -1           14
    teamId_opp  size
0            1    15
1            2    16
2            3    16
3            4    16
4            5    16
5            6    16
6            7    16
7            8    16
8            9    16
9           10    16
10          11    16
11          12    15


- Changing the NotebookApp.iopub_data_rate_limit from 1,000,000 to 10,000,000,000 by using:
    a) jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
        1. Note: As of now, this means that jupyter needs to be launched in this manner everytime i run this
           this code in order to prevent the error. It appears theres away to change the config file though,
           so this can be looked into at another time
- important source: https://stmorse.github.io/journal/espn-fantasy-v3.html

- mMatchup view
    a) schedule dictionary
      1. home and away dictionary

In [58]:
""" This logs into the driver instance, but I'm concerned about the legality of this, so moving on """

# elem = driver.find_element_by_name("username")
# elem.clear()
# elem.send_keys("condelong11@yahoo.com")
# elem.send_keys(Keys.RETURN)

# elem = driver.find_element_by_name("password")
# elem.clear()
# elem.send_keys("Pencil11")
# elem.send_keys(Keys.RETURN)


<selenium.webdriver.remote.webelement.WebElement (session="4d9f9d957944c878900ed0146bd75440", element="0836fc2d-b739-4a5d-978e-5e66e09d7ac9")>
